In [1]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
import os
import sys
import time
import json
import getopt


product_loc = "/home/dzn/Project/d07_optimize/product_json (copy).txt"
coupon_loc  = '/home/dzn/Project/d07_optimize/coupon_json (copy).txt'

#product = pd.DataFrame(json.load(open(product_loc, 'r')))
#coupon = pd.DataFrame(json.load(open(coupon_loc, 'r')))

In [2]:
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")
import numpy as np
#from scipy.optimize import minimize
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")
import json
import getopt

def dotMultiply(a,b):
    """ dot multiply 
    example: A = [1, 2, 3], B = [2, 3, 4], then
    return C = [1*2, 2*3, 3*4] = [2, 6, 12]
    """
    result = []
    for i in xrange(len(a)):
        result.append(a[i]*b[i])
    return result
#***********************************************************************

def formula(a,b):
    """ Formula Expression
    example : A = ['x', 'y', 'z'], B = [1, 2, 3], then 
    return : C = 1*x + 2*y + 3*z
    """
    exp = []
    for i, v in enumerate(a):
        exp.append(str(b[i])+"*"+v)
        if i < len(a) -1 :
            exp.append("+")
    return ' '.join(i for i in exp)

#***********************************************************************
def couponMatchPorduct(coupon_0, product):
    """ Create Product list for each Product """
    selectedproduct = pd.DataFrame()
    dropproduct     = pd.DataFrame()
    category = ['firstCategoryId', 'secondCategoryId', 'thirdCategoryId', 'fouthCategoryId']
    for i in xrange(len(coupon_0['supportCategoryVoList'])):
        item = coupon_0['supportCategoryVoList'][i]
        if item["supportFlag"] :
            for name in item.keys() :
                if name in category :
                    selectedproduct = selectedproduct.append(product[product[name] == item[name]])
        else :
            for name in item.keys() :
                if name in category :
                    dropproduct = dropproduct.append(product[product[name] == item[name]])
    if len(dropproduct) > 0:
        droplist = list(dropproduct.index)
        for i in droplist:
            selectedproduct = selectedproduct.drop(i)   
    
    if len(selectedproduct) > 0 :
        selectedproduct = selectedproduct.drop_duplicates()
        
    return selectedproduct

#********************************************************************
def productMatchCoupon(product_0, coupon):

    category = ['firstCategoryId', 'secondCategoryId', 'thirdCategoryId', 'fouthCategoryId']
    selectcoupon = pd.DataFrame()
    dropcoupon = pd.DataFrame()
    
    for i in xrange(len(coupon)) :
        if coupon.loc[i, "allProductSupportFlag"] == True:
            selectcoupon = selectcoupon.append(coupon.loc[i])
        else :
            for j, name in enumerate(list(pd.DataFrame(product_0 ).index)):
                if name in category :
                    for ii in xrange(len(coupon['supportCategoryVoList'])):
                        item = pd.DataFrame(coupon['supportCategoryVoList'][ii])
                        column = list(item.columns)
                        if name in column :
                            underselectcoupon = item[item[name] == product_0[name]]
                            if len(underselectcoupon) > 0 :
                                aaa = list(underselectcoupon['supportFlag'])
                                if aaa[0] :
                                    selectcoupon = selectcoupon.append(coupon.loc[ii])
                                else :
                                    dropcoupon = dropcoupon.append(coupon.loc[ii])
    
    if len(dropcoupon.index) > 0:
        for j in dropcoupon.index :
            selectcoupon = selectcoupon.drop(j)
    
    if len(selectcoupon) > 0 :
        selectcoupon = selectcoupon.drop_duplicates()
    
    return selectcoupon

#***********************************************************************
def json2DataFrame(jsonpath):
    """ Read Json data into DataFrame """
    jsondata = json.load(open(jsonpath,'r'))
    product  = pd.DataFrame(jsondata)
    return product
#***************************************************************************
def cleaninvest(result) :
    columns = list(set(result.productId))
    result = result.sort_values(by = ["investAmount","couponvalue"], ascending = [True,True])
    for i in columns:
        tmp1 = result[result.productId == i]
        if len(tmp1) >= 2:
            for ii in tmp1.index :
                if len(list(tmp1.couponId[ii])) > 0 :
                    t = ii
                else :
                    result.loc[t, 'investAmount'] += tmp1.loc[ii, 'investAmount']
                    result.loc[t, 'incomeTotal'] += tmp1.loc[ii, 'incomeTotal']
                    result = result.drop(ii)
    result.index = xrange(len(result))
    return result   
#*************************************************************************
class optimizeInvest(object):
    
    def __init__(self, **kwargs):
        
        self.coupon  = pd.DataFrame()
        self.product = pd.DataFrame()
        if 'product' in kwargs.keys():
            product_loc = kwargs['product']
            if not os.path.exists(product_loc) :
                #print("Error : Produc file is not exists !!!")
                sys.exit(11)
            else :
                product  = pd.DataFrame(json.load(open(product_loc,'r')))
                for ii in xrange(len(product)) :
                    if product.loc[ii, 'maxInvestAmount'] == -1 :
                        product.loc[ii,'maxInvestAmount'] = product.loc[ii,'remainInvestAmount']
                        pass
                    
                    if product.loc[ii, "remainInvestAmount"] < product.loc[ii, "minInvestAmount"] :
                        product = product.drop(ii)
                        
                product['realratio'] = 1.0*product.productRatio*product.productDueDays/365
                product['check'] =  product.remainInvestAmount - product.minInvestAmount
                product = product.sort_values(by = ["realratio", "productDueDays", "minInvestAmount", "maxInvestAmount"], ascending=[False, True, True, False])
                product = product[product.check >= 0]
                del product['check']
                product.index = xrange(len(product))
                self.product = product
                                
        if 'coupon' in kwargs.keys():
            coupon_loc = kwargs['coupon']
            if not os.path.exists(coupon_loc):
                #print("Error : Coupon file is not exist !!!")
                sys.exit(11)
            else :
                coupon  = pd.DataFrame(json.load(open(coupon_loc,'r')))
                if len(coupon) > 0:
                    coupon['rate'] = 1.0*coupon.couponAmount/(coupon.couponMinInvestAmount - coupon.couponAmount)
                    coupon['real_start'] = coupon.couponMinInvestAmount - coupon.couponAmount
                    coupon = coupon.sort_values(by = ['couponAmount', 'couponMinInvestAmount'], ascending = [False, True])
                    coupon.index = xrange(len(coupon))
                    self.coupon = coupon             

        
        if len(self.product) == 0:
            #print("Error: The Product list is empty !!!")
            sys.exit(12)
        else :
            pass
    
    def nocoupon(self, needinvest, deadline, product):
        """--- No Coupon Model ---"""
        tmp = 0
        investratio = []
        total = needinvest
        tmp_product = product[product.productDueDays <= deadline]
        new_product =  tmp_product[tmp_product.minInvestAmount <= needinvest]
        new_product.index = xrange(len(new_product))
        prod_cnt    = len(new_product)  
        product     = new_product        
        nocouponresult = pd.DataFrame()
        investproduct = []
        usedcoupon    = []
        couponvalue   = []
        realrate    = []
        realincome  = []
        yearrate = []
        yearincome = []
        investamount = []        
                
        if prod_cnt == 0:
            #print("Warning : No product satisfy the Deadline !!! \n")
            sys.exit(21)
        
        #print new_product
        niter = 0
        maxiter = len(new_product)
        while (needinvest > 0) and len(new_product) > 0 and niter <= maxiter:
            niter += 1
            for i in xrange(len(new_product)):
                if needinvest >= new_product.loc[i, 'minInvestAmount'] :
                    if new_product.loc[i, 'maxInvestAmount'] == -1 :
                        tmpinvest = new_product.loc[i, 'remainInvestAmount']
                    else :
                        tmpinvest = min(new_product.loc[i, 'remainInvestAmount'], new_product.loc[i, 'maxInvestAmount'])
                    
                    if needinvest <= tmpinvest :

                        remainder = needinvest%new_product.loc[i, 'perIncAmount']                        
                        integer   = needinvest - remainder
                        
                        if round(1.0*remainder/new_product.loc[i, 'perIncAmount'], 2) == 0:
                            
                            investproduct.append(new_product.loc[i, 'productId'])
                            usedcoupon.append('')
                            couponvalue.append(0)
                            realrate.append(1.0*new_product.loc[i, 'realratio'])
                            yearrate.append(1.0*new_product.loc[i, 'productRatio'])
                            
                            investamount.append(needinvest)                            
                            realincome.append(needinvest*1.0*new_product.realratio[i])
                            yearincome.append(needinvest*new_product.productRatio[i])
                            needinvest = 0
                            pass
                        else :
                            #print "2222222"
                            if integer >=  new_product.minInvestAmount[i] :
                                investproduct.append(new_product.productId[i])
                                usedcoupon.append('')
                                couponvalue.append(0)
                                realrate.append(1.0*new_product.realratio[i])
                                yearrate.append(1.0*new_product.productRatio[i])
                                
                                investamount.append(integer)
                                realincome.append(integer*1.0*new_product.realratio[i])
                                yearincome.append(integer*new_product.productRatio[i])
                                needinvest = needinvest - integer
                            pass
                        
                        pass
                    else :                        
                        remainder = tmpinvest%new_product.perIncAmount[i]
                        integer   = tmpinvest - remainder
                        
                        if round(1.0*remainder/new_product.perIncAmount[i], 2) == 0:
                            investproduct.append(new_product.productId[i])
                            usedcoupon.append('')
                            couponvalue.append(0)
                            realrate.append(1.0*new_product.realratio[i])
                            yearrate.append(new_product.productRatio[i])
                            
                            investamount.append(tmpinvest)
                            realincome.append(tmpinvest*1.0*new_product.realratio[i])
                            yearincome.append(tmpinvest*new_product.productRatio[i])
                            needinvest = needinvest - tmpinvest
                            pass
                        else :
                            if integer >= new_product.minInvestAmount[i] :
                                
                                investproduct.append(new_product.productId[i])
                                usedcoupon.append('')
                                couponvalue.append(0)
                                realrate.append(1.0*new_product.realratio[i])
                                yearrate.append(new_product.productRatio[i]) 
                                
                                investamount.append(integer)
                                realincome.append(integer*1.0*new_product.realratio[i])
                                yearincome.append(integer*new_product.productRatio[i])
                                needinvest = needinvest - integer
                                pass
                            pass
                        pass
                    new_product = new_product.loc[xrange(i+1, len(new_product)), :]
                    new_product.index = xrange(len(new_product))
                    break
                else :
                    new_product = new_product.loc[xrange(i+1, len(new_product)), :]
                    new_product.index = xrange(len(new_product))
                    break

        
        nocouponresult['productId'] = investproduct
        nocouponresult['investAmount'] = investamount
        nocouponresult['couponId'] = usedcoupon
        nocouponresult['incomeTotal'] = realincome

        if len(investproduct) == 0:
            #print("Warning: No Optimize Investment Satisfied !!! \n")
            sys.exit(12)
                
        return nocouponresult
    
    def usecoupon(self, needinvest, deadline):
        
        """--- Coupon Model ---"""
        topk = 0
        result_coupon = []
        result_nocoupon = []
        unused_coupon   = pd.DataFrame()
        investresult = pd.DataFrame()
        # 1. Choose coupons which satisfy [duedays <= deadline and real_start <= needinvest ]
        #unused_coupon = self.coupon[self.coupon.deadline <= deadline]
        if len(self.coupon) > 0:
            unused_coupon = self.coupon[self.coupon.couponMinInvestAmount <= needinvest]
            unused_coupon.index = xrange(len(unused_coupon))
        # 2. Choose product which satisfy [duedays <= deadline and starting <= needinvest ]
        new_product = self.product[self.product.productDueDays <= deadline]
        if len(new_product) > 0:
            #new_product =  tmp_product[tmp_product.starting <= needinvest]
            new_product.index = xrange(len(new_product))
            pass
        else :
            #print "No Product Satisfy Your Input, Please Use longer Deadline !!!"
            sys.exit(22)
        
        #print new_product
        #print unused_coupon
        couponresult = pd.DataFrame()
        nocouponresult = pd.DataFrame()
        investproduct = []
        usedcoupon    = []
        couponvalue   = []
        realrate    = []
        realincome  = []
        yearrate = []
        yearincome = []
        investamount    = []
        
        newer_product = new_product[new_product.forNewMemberFlag == True]        
        
        # Just for Freshman 
        if len(newer_product) > 0 :
            newer_product = newer_product[newer_product.minInvestAmount <= needinvest]
            # is a freshman,  first select a best product, then select a best coupon for freahman
            
            if len(newer_product) >0 :
                for ij in list(newer_product.index):
                    new_product = new_product.drop(ij)
                    pass
                
                newer_product.index = xrange(len(newer_product))
                
                if len(unused_coupon) > 0:
                    newer_coupon  = productMatchCoupon(newer_product.loc[0], unused_coupon)
                    pass
                else :
                    newer_coupon = pd.DataFrame()
                    pass
                
                if len(newer_coupon) > 0:
                    newer_coupon  = newer_coupon[newer_coupon.couponMinInvestAmount <= min(newer_product.maxInvestAmount[0], newer_product.remainInvestAmount[0])]
                    newer_coupon  = newer_coupon.sort_values(by = ["couponMinInvestAmount", "rate"], ascending=[False, False])
                    newer_coupon.index = xrange(len(newer_coupon))
                    #print "##############"
                    #print newer_coupon
                    if len(newer_coupon) > 0:
                        for step in xrange(len(newer_coupon)) :
                            if newer_coupon.couponMinInvestAmount[step] <= needinvest :
                                investproduct.append(newer_product.productId[0])
                                usedcoupon.append(newer_coupon.couponId[step])
                                couponvalue.append(newer_coupon.couponAmount[step])
                                realrate.append(1.0*newer_product.realratio[0])
                                yearrate.append(newer_product.productRatio[0])
                                useed_coupon = unused_coupon[unused_coupon.couponId == newer_coupon.couponId[step]].index
                                unused_coupon   = unused_coupon.drop(useed_coupon[0])
                                unused_coupon.index = xrange(len(unused_coupon))
                                
                                tmpinvest = min(newer_product.maxInvestAmount[0], newer_product.remainInvestAmount[0])
                                if tmpinvest <= needinvest :
                                    #integer   = round(newer_product.maxInvestAmount[0]/newer_product.perIncAmount[0],0)
                                    remainder = tmpinvest%newer_product.perIncAmount[0]
                                    integer =  tmpinvest- remainder
                                    if round(1.0*remainder/newer_product.perIncAmount[0], 2) == 0:
                                        investamount.append(tmpinvest)
                                        realincome.append(tmpinvest*1.0*newer_product.realratio[0] + newer_coupon.couponAmount[step])
                                        yearincome.append(tmpinvest*newer_product.productRatio[0] + newer_coupon.couponAmount[step])
                                        needinvest = needinvest - tmpinvest
                                        pass
                                    else :
                                        if integer >= newer_product.minInvestAmount[0] :
                                            investamount.append(integer)
                                            realincome.append(integer*1.0*newer_product.realratio[0] + newer_coupon.couponAmount[step])
                                            yearincome.append(integer*newer_product.productRatio[0] + newer_coupon.couponAmount[step])
                                            needinvest = needinvest - integer
                                        pass
                                    pass
                                else :
                                    remainder = needinvest%newer_product.perIncAmount[0]
                                    integer   = needinvest - remainder
                                    if round(1.0*remainder/newer_product.perIncAmount[0], 2) == 0:
                                        investamount.append(needinvest)
                                        realincome.append(needinvest*1.0*newer_product.realratio[0] + newer_coupon.couponAmount[step])
                                        yearincome.append(needinvest*newer_product.productRatio[0] + newer_coupon.couponAmount[step])
                                        needinvest    = 0
                                        pass
                                    else :
                                        if integer >= newer_product.minInvestAmount[0] :
                                            investamount.append(integer)
                                            realincome.append(integer*1.0*newer_product.realratio[0] + newer_coupon.couponAmount[step])
                                            yearincome.append(integer*newer_product.productRatio[0] + newer_coupon.couponAmount[step])
                                            needinvest = needinvest - integer
                                        pass
                                    pass
                                break
                            pass
                        pass
                    else :
                        #print '333333333'
                        investproduct.append(newer_product.loc[0,'productId'])
                        usedcoupon.append('')
                        couponvalue.append(0)
                        realrate.append(1.0*newer_product.loc[0, 'realratio'])
                        yearrate.append(newer_product.loc[0, 'productRatio'])
                        
                        tmp_invest = min(newer_product.loc[0, 'maxInvestAmount'], newer_product.loc[0, "remainInvestAmount"])
                        
                        if tmp_invest <= needinvest:
                            #integer   = round(tmp_invest/newer_product.perIncAmount[0],0)
                            remainder = tmp_invest%newer_product.loc[0,'perIncAmount']
                            integer   = tmp_invest - remainder
                            
                            if round(1.0*remainder/newer_product.perIncAmount[0], 2) == 0:
                                investamount.append(tmp_invest)
                                realincome.append(tmp_invest*1.0*newer_product.realratio[0])
                                yearincome.append(tmp_invest*newer_product.productRatio[0])
                                needinvest = needinvest - tmp_invest
                                pass
                            else :
                                if integer >= newer_product.minInvestAmount[0] :
                                    investamount.append(integer)
                                    realincome.append(integer*1.0*newer_product.realratio[0])
                                    yearincome.append(integer*newer_product.productRatio[0])
                                    needinvest = needinvest - integer
                                pass
                            pass
                        else :
                            remainder = needinvest%newer_product.perIncAmount[0]
                            integer   = needinvest - remainder
                            if round(1.0*remainder/newer_product.perIncAmount[0], 2) == 0:
                                investamount.append(needinvest)
                                realincome.append(needinvest*1.0*newer_product.realratio[0])
                                yearincome.append(needinvest*newer_product.productRatio[0])
                                needinvest    = 0
                                pass
                            else :
                                if integer >= newer_product.minInvestAmount[0] :
                                    investamount.append(integer)
                                    realincome.append(integer*1.0*newer_product.realratio[0])
                                    yearincome.append(integer*newer_product.productRatio[0])
                                    needinvest = needinvest - integer
                                pass
                            pass
                        pass
                    pass
                else :
                    investproduct.append(newer_product.productId[0])
                    usedcoupon.append('')
                    couponvalue.append(0)
                    realrate.append(1.0*newer_product.realratio[0])
                    yearrate.append(newer_product.productRatio[0])
                    #print '333333333'
                    tmp_invest = min(newer_product.maxInvestAmount[0], newer_product.remainInvestAmount[0])  
                    if tmp_invest <= needinvest:
                        remainder = tmp_invest%newer_product.perIncAmount[0]
                        integer   = tmp_invest - remainder                        
                        if round(1.0*remainder/newer_product.perIncAmount[0], 2) == 0:
                            investamount.append(tmp_invest)
                            realincome.append(tmp_invest*1.0*newer_product.realratio[0])
                            yearincome.append(tmp_invest*newer_product.productRatio[0])
                            needinvest    = needinvest - tmp_invest 
                            pass
                        else :
                            if integer >= newer_product.minInvestAmount[0] :
                                investamount.append(integer)
                                realincome.append(integer*1.0*newer_product.realratio[0])
                                yearincome.append(integer*newer_product.productRatio[0])
                                needinvest = needinvest - integer
                            pass
                        pass
                    else :
                        remainder = needinvest%newer_product.perIncAmount[0]
                        integer   = needinvest - remainder                        
                        if round(1.0*remainder/newer_product.perIncAmount[0], 2) == 0:
                            investamount.append(needinvest)
                            realincome.append(needinvest*1.0*newer_product.realratio[0])
                            yearincome.append(needinvest*newer_product.productRatio[0])
                            needinvest    = 0
                            pass
                        else :
                            if integer >= newer_product.minInvestAmount[0] :
                                investamount.append(integer)
                                realincome.append(integer*1.0*newer_product.realratio[0])
                                yearincome.append(integer*newer_product.productRatio[0])
                                needinvest = needinvest - integer
                            pass
                        pass
                    pass
                pass
            pass
        
        iter = 0
        maxiter = len(unused_coupon)
        while ( len(unused_coupon) > 0 and iter <= maxiter) :
            iter += 1
            coupon_0 = unused_coupon.loc[0]

            if "allProductSupportFlag" in list(pd.DataFrame(coupon_0).index) :
                if coupon_0['allProductSupportFlag'] == True :
                    tmp_product = new_product
                    pass
                else :
                    tmp_product = couponMatchPorduct(coupon_0, new_product)
                    pass
                pass
            else :
                tmp_product = couponMatchPorduct(coupon_0, new_product)                     
                pass
            
            #tmp_product = tmp_product[tmp_product.minInvestAmount >= (unused_coupon.loc[0,'couponMinInvestAmount'] - unused_coupon.loc[0,'couponAmount'])]
            tmp_product = tmp_product[tmp_product.minInvestAmount <= needinvest]
            tmp_product = tmp_product[tmp_product.maxInvestAmount >= unused_coupon.loc[0,'couponMinInvestAmount']]

            if len(tmp_product) > 0 :

                if len(unused_coupon) == 1:
                    tmp_product = tmp_product.sort_values(by = ["realratio","minInvestAmount"], ascending=[False, False])
                    tmp_product.index = xrange(len(tmp_product))
                    pass
                else :
                    tmp_product = tmp_product.sort_values(by = ["realratio","minInvestAmount"], ascending=[False, True])
                    tmp_product.index = xrange(len(tmp_product))
                    pass
                
                for i in xrange(len(tmp_product)) :
                    real_invest = max(unused_coupon.couponMinInvestAmount[0], tmp_product.minInvestAmount[i])
                    if needinvest >= real_invest :

                        if len(unused_coupon) == 1:
                            if needinvest - 2*real_invest > 0:
                                remainder = real_invest%tmp_product.perIncAmount[i]
                                integer   = real_invest - remainder
                                if round(1.0*remainder/tmp_product.perIncAmount[i], 2) == 0:
                                    if real_invest <= tmp_product.maxInvestAmount[i]:
                                        investamount.append(real_invest)
                                        realincome.append(real_invest*1.0*tmp_product.realratio[i] + unused_coupon.couponAmount[0])
                                        yearincome.append(real_invest*tmp_product.productRatio[i] + unused_coupon.couponAmount[0])
                                        needinvest = needinvest - real_invest
                                        
                                        investproduct.append(tmp_product.productId[i])
                                        usedcoupon.append(unused_coupon.couponId[0])
                                        couponvalue.append(unused_coupon.couponAmount[0])
                                        realrate.append(1.0*tmp_product.realratio[i])
                                        yearrate.append(tmp_product.productRatio[i])
                                        
                                        used_product = new_product[new_product.productId == tmp_product.loc[i, 'productId']].index
                                        new_product.loc[ used_product[0], "maxInvestAmount"] -= real_invest

                                        pass
                                    pass
                                pass
                            else :
                                
                                tmp_invest = min(needinvest, tmp_product.maxInvestAmount[i])
                                
                                if tmp_invest >= real_invest :
                                    remainder = tmp_invest%tmp_product.perIncAmount[i]
                                    integer   = tmp_invest - remainder
                                    
                                    if round(1.0*remainder/tmp_product.perIncAmount[i], 2) == 0:
                                        investamount.append(tmp_invest)
                                        realincome.append(tmp_invest*1.0*tmp_product.realratio[i] + unused_coupon.couponAmount[0])
                                        yearincome.append(tmp_invest*tmp_product.productRatio[i] + unused_coupon.couponAmount[0])
                                        needinvest = needinvest - tmp_invest
                                        
                                        investproduct.append(tmp_product.productId[i])
                                        usedcoupon.append(unused_coupon.couponId[0])
                                        couponvalue.append(unused_coupon.couponAmount[0])
                                        realrate.append(1.0*tmp_product.realratio[i])
                                        yearrate.append(tmp_product.productRatio[i])
                                        
                                        used_product = new_product[new_product.productId == tmp_product.loc[i, 'productId']].index
                                        new_product.loc[ used_product[0], "maxInvestAmount"] -= tmp_invest

                                        pass
                                    else :                                        
                                        if real_invest <= integer:
                                            investamount.append(integer)
                                            realincome.append(integer*1.0*tmp_product.loc[i, 'realratio'] + unused_coupon.loc[0, 'couponAmount'])
                                            yearincome.append(integer*tmp_product.loc[i, 'productRatio'] + unused_coupon.loc[0, 'couponAmount'])
                                            needinvest = needinvest - integer
                                            
                                            investproduct.append(tmp_product.loc[i, 'productId'])
                                            usedcoupon.append(unused_coupon.loc[0, 'couponId'])
                                            couponvalue.append(unused_coupon.loc[0, 'couponAmount'])
                                            realrate.append(1.0*tmp_product.loc[i, 'realratio'])
                                            yearrate.append(tmp_product.loc[i, 'productRatio'])
                                            
                                            used_product = new_product[new_product.productId == tmp_product.loc[i,'productId']].index
                                            new_product.loc[ used_product[0], "maxInvestAmount"] -= integer
                                            pass
                                        pass
                                    pass
                                pass
                            pass
                        else :                            
                            remainder = real_invest%tmp_product.perIncAmount[i]
                            integer   = real_invest - remainder
                            
                            if round(1.0*remainder/tmp_product.perIncAmount[i], 2) == 0:
                                if real_invest <= tmp_product.loc[i, 'maxInvestAmount'] :
                                    investamount.append(real_invest)
                                    realincome.append(real_invest*1.0*tmp_product.realratio[i] + unused_coupon.couponAmount[0])
                                    yearincome.append(real_invest*tmp_product.productRatio[i] + unused_coupon.couponAmount[0])
                                    needinvest = needinvest - real_invest
                                    
                                    used_product = new_product[new_product.productId == tmp_product.loc[i,'productId']].index
                                    new_product.loc[ used_product[0], "maxInvestAmount"] -= real_invest
                                    
                                    investproduct.append(tmp_product.productId[i])
                                    usedcoupon.append(unused_coupon.couponId[0])
                                    couponvalue.append(unused_coupon.couponAmount[0])
                                    realrate.append(1.0*tmp_product.realratio[i])
                                    yearrate.append(tmp_product.productRatio[i])
                                    pass
                                pass
                            pass
                        unused_coupon = unused_coupon.drop(0)
                        unused_coupon.index = xrange(len(unused_coupon))
                        new_product.loc[:,'check'] = new_product.maxInvestAmount - new_product.minInvestAmount
                        new_product = new_product[new_product.check >= 0]
                        del new_product['check']
                        new_product.index = xrange(len(new_product))
                        break
                        pass
                    else :
                        if i < (len(tmp_product) - 1) :
                            pass
                        else :
                            unused_coupon = unused_coupon.drop(0)
                            unused_coupon.index = xrange(len(unused_coupon))
                            new_product['check'] = new_product.maxInvestAmount - new_product.minInvestAmount
                            new_product = new_product[new_product.check >= 0]
                            del new_product['check']
                            new_product.index = xrange(len(new_product))
                            break
                            pass
                        pass
                    pass
                pass            
            else :
                unused_coupon = unused_coupon.drop(0)
                unused_coupon.index = xrange(len(unused_coupon))
                pass

        new_product['check'] = new_product.maxInvestAmount - new_product.minInvestAmount
        new_product = new_product[new_product.check >= 0]
        del new_product['check']
        new_product.index = xrange(len(new_product))
        
        if len(new_product) > 0:
            if needinvest >= min(new_product.minInvestAmount):
                nocouponresult = self.nocoupon(needinvest, deadline, new_product)

        couponresult['productId'] = investproduct
        couponresult['investAmount'] = investamount
        couponresult['couponId'] = usedcoupon
        couponresult['incomeTotal'] = realincome
        couponresult['couponvalue'] = couponvalue
        investment = couponresult.append(nocouponresult)
        investment.index = xrange(len(investment))
        
        investment = cleaninvest(investment)
        
        jsonresult = ''
        
        if len(investment) > 0:
            for tt in xrange(len(investment)):
                if tt == 0 :
                    jsonresult = pd.DataFrame.to_json(pd.DataFrame(investment.loc[tt]))
                    pass
                else :
                    tmpjosn = pd.DataFrame.to_json(pd.DataFrame(investment.loc[tt]))
                    jsonresult = jsonresult[:-1] +',' +tmpjosn[1:]
                    pass
                pass
            pass

        return investment
    
    def invest(self, needinvest, deadline):
        
        """--- Optimize Invest --- 
        IF User has coupons, then Coupon Model
        IF User doesn't have coupon, then use No Coupon model
        """
        
        result = self.usecoupon(needinvest, deadline)
        return result


In [3]:
optime = optimizeInvest(product=product_loc, coupon=coupon_loc)

In [4]:
start_CPU = time.clock()
t1 = time.time()
result = optime.invest(888800, 765)
end_CPU = time.clock()
t2 = time.time()
#end_CPU - start_CPU
#t2 - t1

In [5]:
result

,couponId,couponvalue,incomeTotal,investAmount,productId
0,0acdc3f2b82045f9bd27d0c2ae07e555,10,3738.219178,10000,714
1,308efd57b144482eb3a4e178cea08b80,10,3738.219178,10000,714
2,eb8cfc2378cf44ad9f5a607ceae42ae4,10,3738.219178,10000,714
3,bd65bbc4e00944ec9e20f1e275105a13,100,3828.219178,10000,714
4,e9e85e19bc644d718f1cd1b59ef6ac00,5000,135040.284932,348800,714
5,40910c095ff04a6593a9874944ad95e4,50000,236410.958904,500000,714


In [26]:
#product_loc = "/home/dzn/Project/d07_optimize/products_20160620.csv"
#coupon_loc  = '/home/dzn/Project/d07_optimize/coupons_20160620.csv'

In [10]:
product = pd.DataFrame(json.load(open(product_loc,'r')))
product = product.sort_values(by = ["productRatio", "productDueDays", "minInvestAmount", 'maxInvestAmount'], ascending=[False, True, True, False])
product.index = range(len(product))
product

,firstCategoryId,forNewMemberFlag,maxInvestAmount,minInvestAmount,perIncAmount,productDueDays,productId,productRatio,remainInvestAmount,secondCategoryId,thirdCategoryId
0,2,False,8000,100,100,270,523,0.2000,200000,20,21
1,2,False,-1,100,100,180,521,0.1800,12000,4,8
2,2,False,900,100,100,270,519,0.1800,99400,20,21
3,2,False,-1,100,1000,365,510,0.1700,100000,3,6
4,2,False,-1,100,1000,365,509,0.1700,100000,3,7
5,2,False,-1,100,1000,365,508,0.1600,10000,47,66
6,2,False,-1,100,1000,365,517,0.1600,100000000,5,25
7,2,False,-1,100,1000,365,515,0.1600,100000000,51,92
8,2,False,-1,100,1000,365,513,0.1600,100000,50,85
9,2,False,-1,100,1000,365,511,0.1600,100000,3,6


In [11]:
coupon = pd.DataFrame(json.load(open(coupon_loc,'r')))
coupon['rate'] = 1.0*coupon.couponAmount/(coupon.couponMinInvestAmount-coupon.couponAmount)
coupon['real_start'] = coupon.couponMinInvestAmount-coupon.couponAmount
coupon = coupon.sort_values(by=['rate', 'real_start'],  ascending = [False, True])
coupon

,allProductSupportFlag,couponAmount,couponId,couponMinInvestAmount,rate,real_start
0,True,10,8e3a7fed24f443f6a7d5547060be5e6b,100,0.111111,90
10,True,10,839a51567e9f41d4ba26ac3825ffc4da,100,0.111111,90
1,True,100,5cc5b346c5f442d3aef8e8dd840c6241,1000,0.111111,900
11,True,100,342a2845fa27414ab0f878be0d4ad906,1000,0.111111,900
12,True,100,d369da2bc1a240e8a432b058f88a26ce,1000,0.111111,900
13,True,100,504de7d3f10d42499b981aa7bfa0a954,1000,0.111111,900
14,True,100,212d8458484b4ecc8231fe20dce61a47,1000,0.111111,900
15,True,100,e2be2f598fb645a38b1a6b76178ec660,1000,0.111111,900
16,True,100,2506b9d988074663a3b79fdba846a01b,1000,0.111111,900
17,True,100,2bf038f02d724ca186c4c7e2c026be49,1000,0.111111,900


In [33]:
def productMatchCoupon(product_0, coupon):

    category = ['firstCategoryId', 'secondCategoryId', 'thirdCategoryId', 'fouthCategoryId']
    selectcoupon = pd.DataFrame()
    dropcoupon = pd.DataFrame()
    
    for i in xrange(len(coupon)) :
        if coupon.loc[i, "allProductSupportFlag"] == True:
            selectcoupon = selectcoupon.append(coupon.loc[i])
        else :
            for j, name in enumerate(list(pd.DataFrame(product_0 ).index)):
                if name in category :
                    for ii in xrange(len(coupon['supportCategoryVoList'])):
                        item = pd.DataFrame(coupon['supportCategoryVoList'][ii])
                        column = list(item.columns)
                        if name in column :
                            underselectcoupon = item[item[name] == product_0[name]]
                            if len(underselectcoupon) > 0 :
                                aaa = list(underselectcoupon['supportFlag'])
                                if aaa[0] :
                                    selectcoupon = selectcoupon.append(coupon.loc[ii])
                                else :
                                    dropcoupon = dropcoupon.append(coupon.loc[ii])
    
    if len(dropcoupon.index) > 0:
        for j in dropcoupon.index :
            selectcoupon = selectcoupon.drop(j)
    
    if len(selectcoupon) > 0 :
        selectcoupon = selectcoupon.drop_duplicates()
    
    return selectcoupon


In [34]:
productMatchCoupon( product.loc[5],coupon)

   allProductSupportFlag  couponAmount                          couponId  \
0                      1            10  0acdc3f2b82045f9bd27d0c2ae07e555   
1                      1            10  308efd57b144482eb3a4e178cea08b80   
2                      1          1000  3153ffe1638a4f8f82afb96a90374d01   
3                      1          5000  e9e85e19bc644d718f1cd1b59ef6ac00   
4                      1         50000  40910c095ff04a6593a9874944ad95e4   
5                      1        100000  1bf2687581a84b798db0d09e87d07c9d   
6                      1           100  bd65bbc4e00944ec9e20f1e275105a13   
7                      1            10  eb8cfc2378cf44ad9f5a607ceae42ae4   

   couponMinInvestAmount      rate  real_start  
0                    100  0.111111          90  
1                    100  0.111111          90  
2                  10000  0.111111        9000  
3                  50000  0.111111       45000  
4                 500000  0.111111      450000  
5                1000

,allProductSupportFlag,couponAmount,couponId,couponMinInvestAmount,rate,real_start
0,1,10,0acdc3f2b82045f9bd27d0c2ae07e555,100,0.111111,90
1,1,10,308efd57b144482eb3a4e178cea08b80,100,0.111111,90
2,1,1000,3153ffe1638a4f8f82afb96a90374d01,10000,0.111111,9000
3,1,5000,e9e85e19bc644d718f1cd1b59ef6ac00,50000,0.111111,45000
4,1,50000,40910c095ff04a6593a9874944ad95e4,500000,0.111111,450000
5,1,100000,1bf2687581a84b798db0d09e87d07c9d,1000000,0.111111,900000
6,1,100,bd65bbc4e00944ec9e20f1e275105a13,1000,0.111111,900
7,1,10,eb8cfc2378cf44ad9f5a607ceae42ae4,100,0.111111,90


In [31]:
[i for i in xrange(10)]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]